In [3]:
from typing import Annotated, Literal
import os 
from autogen import ConversableAgent

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
config_list = [{
    "model": "gpt_4o_mini_0513",
    "base_url": "https://aoai-eaom-poc-gpt4o.openai.azure.com/",
    "api_type": "azure",
    "api_version": "2024-04-01-preview",
    "api_type": "azure",
    "api_key": os.getenv("AZURE_OPENAI_API_KEY")
    }]
llm_config = {"config_list": config_list, "temperature": 0.3,"cache_seed": 42, "timeout": 120,}

In [6]:
Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [7]:
# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

In [8]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_roa56Dc58Hx261lI3q4aokl4): calculator *****
Arguments: 
{"a": 13312, "b": 200, "operator": "/"}
***************************************************************************
***** Suggested tool call (call_bhnlJPHY5JklpJD5HmASrZed): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_roa56Dc58Hx261lI3q4aokl4) *****
66
**********************************************************************

---------------------------------------

In [11]:
chat_result.chat_history

[{'content': 'What is (44232 + 13312 / (232 - 32)) * 5?', 'role': 'assistant'},
 {'tool_calls': [{'id': 'call_roa56Dc58Hx261lI3q4aokl4',
    'function': {'arguments': '{"a": 13312, "b": 200, "operator": "/"}',
     'name': 'calculator'},
    'type': 'function'},
   {'id': 'call_bhnlJPHY5JklpJD5HmASrZed',
    'function': {'arguments': '{"a": 232, "b": 32, "operator": "-"}',
     'name': 'calculator'},
    'type': 'function'}],
  'content': None,
  'role': 'assistant'},
 {'content': '66\n\n200',
  'tool_responses': [{'tool_call_id': 'call_roa56Dc58Hx261lI3q4aokl4',
    'role': 'tool',
    'content': '66'},
   {'tool_call_id': 'call_bhnlJPHY5JklpJD5HmASrZed',
    'role': 'tool',
    'content': '200'}],
  'role': 'tool'},
 {'tool_calls': [{'id': 'call_2e4xqtK1XE3yTFzY7sTuCzHF',
    'function': {'arguments': '{"a":44232,"b":66,"operator":"+"}',
     'name': 'calculator'},
    'type': 'function'}],
  'content': None,
  'role': 'assistant'},
 {'content': '44298',
  'tool_responses': [{'tool_c

In [13]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]

# Using `Pydantic`

In [14]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: Annotated[int, Field(description="The first number.")]
    b: Annotated[int, Field(description="The second number.")]
    operator: Annotated[Operator, Field(description="The operator.")]


def calculator(input: Annotated[CalculatorInput, "Input to the calculator."]) -> int:
    if input.operator == "+":
        return input.a + input.b
    elif input.operator == "-":
        return input.a - input.b
    elif input.operator == "*":
        return input.a * input.b
    elif input.operator == "/":
        return int(input.a / input.b)
    else:
        raise ValueError("Invalid operator")

In [21]:
CalculatorInput.model_json_schema()

{'properties': {'a': {'description': 'The first number.',
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'The second number.', 'title': 'B', 'type': 'integer'},
  'operator': {'description': 'The operator.',
   'enum': ['+', '-', '*', '/'],
   'title': 'Operator',
   'type': 'string'}},
 'required': ['a', 'b', 'operator'],
 'title': 'CalculatorInput',
 'type': 'object'}

In [22]:
assistant.register_for_llm(name="calculator", description="A calculator tool that accepts nested expression as input")(
    calculator
)
user_proxy.register_for_execution(name="calculator")(calculator)

c:\Users\AMD\anaconda3\envs\pyautogen_311\Lib\site-packages\autogen\agentchat\conversable_agent.py:2490: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
c:\Users\AMD\anaconda3\envs\pyautogen_311\Lib\site-packages\autogen\agentchat\conversable_agent.py:2409: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


<function __main__.calculator(input: typing.Annotated[__main__.CalculatorInput, 'Input to the calculator.']) -> int>

In [23]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A calculator tool that accepts nested expression as input',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'input': {'properties': {'a': {'description': 'The first number.',
        'title': 'A',
        'type': 'integer'},
       'b': {'description': 'The second number.',
        'title': 'B',
        'type': 'integer'},
       'operator': {'description': 'The operator.',
        'enum': ['+', '-', '*', '/'],
        'title': 'Operator',
        'type': 'string'}},
      'required': ['a', 'b', 'operator'],
      'title': 'CalculatorInput',
      'type': 'object',
      'description': 'Input to the calculator.'}},
    'required': ['input']}}}]

In [24]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_6XxSXeEsgXZuKzKcEvmPEvZ6): calculator *****
Arguments: 
{"input": {"a": 1423, "b": 123, "operator": "-"}}
***************************************************************************
***** Suggested tool call (call_kyAqDvvM65pMPm86jimxsMhG): calculator *****
Arguments: 
{"input": {"a": 32, "b": 23, "operator": "+"}}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_6XxSXeEsgXZuKzKcEvmPEvZ6) *****
1300
**********************************************************************

-----------------